In [16]:
import os

In [17]:
%pwd

'c:\\Users\\GEN-AI\\End-to-End-machine-project-with-mlflow-'

In [ ]:
#os.chdir("../")

In [18]:
%pwd

'c:\\Users\\GEN-AI\\End-to-End-machine-project-with-mlflow-'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
        

    
    model_name: str
    alpha : float
    l1_ratio : float
    target_column: str


In [32]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManger:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepth  = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params  = read_yaml(params_filepth)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self)  -> ModelTrainerConfig:
          config = self.config.model_trainer
          params  = self.params.ElasticNet
          schema = self.schema.TARGET_COLUMN

          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
                root_dir = config.root_dir,
                train_data_path = config.train_data_path,
                test_data_path = config.test_data_path,
                model_name = config.model_name,
                alpha = params.alpha,
                l1_ratio  = params.l1_ratio,
                target_column = schema.name
          )

          return model_trainer_config
        

    

In [33]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from mlProject import logger
import joblib


In [39]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config  =config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        

        train_x = train_data.drop([self.config.target_column],axis=1)
        test_y = test_data.drop([self.config.target_column],axis=1)
        train_y =train_data[[self.config.target_column]]
        test_y  = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha = self.config.alpha,l1_ratio = self.config.l1_ratio,random_state=43)
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

In [40]:
try:
    config = ConfigurationManger()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-06-19 16:32:25,292: INFO: common: YAML file config\config.yaml loaded successfully]
[2025-06-19 16:32:25,314: INFO: common: YAML file params.yaml loaded successfully]
[2025-06-19 16:32:25,322: INFO: common: YAML file schema.yaml loaded successfully]
[2025-06-19 16:32:25,330: INFO: common: Created directory at: artifacts]
[2025-06-19 16:32:25,334: INFO: common: Created directory at: artifacts/model_trainer]
